# DSWE Analysis

This code is used to analyze DSWE inundation assets generated using the 'Monthly Landsat DSWE Generator'script. The user provides folders to the DSWE and QC folders. It outputs a .csv of statistics including the DSWE pixel and area coverage by value, as well as QC statistics for each QC product. 

DSWE Methodology: Jones, J.W., 2019. Improved Automated Detection of Subpixel-Scale Inundation—Revised Dynamic Surface Water Extent (DSWE) Partial Surface Water Tests. Remote Sensing 11, 374. https://doi.org/10.3390/rs11040374

Landsat Collection 2: Earth Resources Observation and Science (EROS) Center. (2020). Landsat 8-9 Operational Land Imager / Thermal Infrared Sensor Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9OGBGM6. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 7 Enhanced Thematic Mapper Plus Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9C7I13B. Earth Resources Observation and Science (EROS) Center. (2020). Landsat 4-5 Thematic Mapper Level-2, Collection 2 [dataset]. U.S. Geological Survey. https://doi.org/10.5066/P9IAXOVV.

Google Earth Engine: Gorelick, N., Hancher, M., Dixon, M., Ilyushchenko, S., Thau, D., Moore, R., 2017. Google Earth Engine: Planetary-scale geospatial analysis for everyone. Remote Sensing of Environment, Big Remotely Sensed Data: tools, applications and experiences 202, 18–27. https://doi.org/10.1016/j.rse.2017.06.031

Author: James (Huck) Rees, PhD Student, UC Santa Barbara Geography

Date: August 18, 2025

## Import packages

In [1]:
import ee
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
from tqdm import tqdm

ee.Initialize()

## Initialize functions for Landsat DSWE

In [2]:
def generate_monthly_ids(folder_prefix, prefix_label, start_year, start_month, end_year, end_month):
    start_date = datetime(start_year, start_month, 1)
    end_date = datetime(end_year, end_month, 1)
    image_ids = []

    while start_date <= end_date:
        year = start_date.year
        month = f"{start_date.month:02d}"
        img_id = f"{folder_prefix}/{prefix_label}_{year}_{month}"
        date_str = f"{year}-{month}"
        image_ids.append((img_id, date_str))
        start_date += relativedelta(months=1)

    return image_ids

def compute_dswe_stats(image_id, date_str, scale=30):
    stats = {'date': date_str}
    classes = [0, 1, 2, 3, 4]

    try:
        image = ee.Image(image_id).select('dswe')

        for cls in classes:
            count = image.eq(cls).reduceRegion(
                reducer=ee.Reducer.sum(),
                scale=scale,
                maxPixels=1e13
            ).get('dswe')
            area_km2 = ee.Number(count).multiply(scale * scale).divide(1e6)
            stats[f'count_{cls}'] = count.getInfo() if count else 0
            stats[f'area_km2_{cls}'] = area_km2.getInfo() if area_km2 else 0

        water_mask = image.gt(0)
        water_count = water_mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            scale=scale,
            maxPixels=1e13
        ).get('dswe')
        water_area = ee.Number(water_count).multiply(scale * scale).divide(1e6)

        stats['count_1to4'] = water_count.getInfo() if water_count else 0
        stats['area_km2_1to4'] = water_area.getInfo() if water_area else 0

        return stats

    except Exception:
        print(f"⚠️ DSWE image not found or failed to process: {image_id}")
        return None

def compute_qc_stats(qc_image_id, scale=30):
    try:
        image = ee.Image(qc_image_id).select('expansion_mask')
    except Exception:
        print(f"⚠️ QC image not found or failed to process: {qc_image_id}")
        return {}

    stats = {}

    for value in range(7):  # QC values 0–6
        mask = image.eq(value)
        count = mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            scale=scale,
            maxPixels=1e13
        ).get('expansion_mask')
        area_km2 = ee.Number(count).multiply(scale * scale).divide(1e6)
        stats[f'qc_count_{value}'] = count.getInfo() if count else 0
        stats[f'qc_area_km2_{value}'] = area_km2.getInfo() if area_km2 else 0

    # Total stats
    pixel_count = image.reduceRegion(
        reducer=ee.Reducer.count(),
        scale=scale,
        maxPixels=1e13
    ).get('expansion_mask')

    mean_value = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        scale=scale,
        maxPixels=1e13
    ).get('expansion_mask')

    total_area = ee.Number(pixel_count).multiply(scale * scale).divide(1e6)

    stats['qc_pixel_count'] = pixel_count.getInfo() if pixel_count else 0
    stats['qc_area_km2'] = total_area.getInfo() if total_area else 0
    stats['qc_avg_value'] = mean_value.getInfo() if mean_value else None

    return stats

def batch_process_LS_dswe_with_qc(dswe_folder, qc_folder, start_year, start_month, end_year, end_month, output_csv_path):
    image_entries = generate_monthly_ids(dswe_folder, 'DSWE', start_year, start_month, end_year, end_month)
    all_stats = []

    for dswe_id, date_str in image_entries:
        print(f"📅 Processing {date_str}...")
        dswe_stats = compute_dswe_stats(dswe_id, date_str)
        if dswe_stats:
            year, month = date_str.split('-')
            qc_id = f"{qc_folder}/QC_{year}_{month}"
            qc_stats = compute_qc_stats(qc_id)
            all_stats.append(dswe_stats | qc_stats)

    df = pd.DataFrame(all_stats)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
    df = df.sort_values(by='date')

    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ CSV saved to: {output_csv_path}")

## Initialize functions for annual Landsat DSWE JASO masks

In [4]:
def generate_annual_ids(folder_prefix, start_year, end_year):
    """Generate annual DSWE image IDs for JASO seasonal composites."""
    image_ids = []
    for year in range(start_year, end_year + 1):
        img_id = f"{folder_prefix}/DSWE_{year}_JASO"
        image_ids.append((img_id, year))
    return image_ids

def compute_dswe_stats(image_id, year, scale=30):
    """Compute pixel counts and areas for each DSWE class."""
    stats = {'year': year}
    classes = [0, 1, 2, 3, 4]
    
    try:
        image = ee.Image(image_id).select('dswe')
        
        # Compute stats for each individual class
        for cls in classes:
            count = image.eq(cls).reduceRegion(
                reducer=ee.Reducer.sum(),
                scale=scale,
                maxPixels=1e13
            ).get('dswe')
            area_km2 = ee.Number(count).multiply(scale * scale).divide(1e6)
            stats[f'count_{cls}'] = count.getInfo() if count else 0
            stats[f'area_km2_{cls}'] = area_km2.getInfo() if area_km2 else 0
        
        # Compute aggregate water stats (classes 1-4)
        water_mask = image.gt(0)
        water_count = water_mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            scale=scale,
            maxPixels=1e13
        ).get('dswe')
        water_area = ee.Number(water_count).multiply(scale * scale).divide(1e6)
        stats['count_1to4'] = water_count.getInfo() if water_count else 0
        stats['area_km2_1to4'] = water_area.getInfo() if water_area else 0
        
        return stats
        
    except Exception as e:
        print(f"⚠️ DSWE image not found or failed to process: {image_id}")
        print(f"   Error: {e}")
        return None

def batch_process_annual_dswe(folder_prefix, start_year, end_year, output_csv_path):
    """Process annual JASO DSWE products and export statistics to CSV."""
    image_entries = generate_annual_ids(folder_prefix, start_year, end_year)
    all_stats = []
    
    for dswe_id, year in image_entries:
        print(f"📅 Processing {year}...")
        dswe_stats = compute_dswe_stats(dswe_id, year)
        
        if dswe_stats:
            all_stats.append(dswe_stats)
    
    # Create DataFrame and save
    df = pd.DataFrame(all_stats)
    df = df.sort_values(by='year')
    
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ CSV saved to: {output_csv_path}")
    print(f"   Processed {len(all_stats)} years successfully")
    
    return df

## Initialize functions for Sentinel-2 DSWE

In [3]:
# Monthly ID Generator
def generate_s2_monthly_ids(dswe_folder, start_year, start_month, end_year, end_month):
    start_date = datetime(start_year, start_month, 1)
    end_date = datetime(end_year, end_month, 1)
    image_ids = []

    while start_date <= end_date:
        year = start_date.year
        month = f"{start_date.month:02d}"
        image_id = f"{dswe_folder}/DSWE_{year}_{month}"
        date_str = f"{year}-{month}"
        image_ids.append((image_id, date_str))
        start_date += relativedelta(months=1)

    return image_ids

# Pixel Count and Area
def calculate_area_from_class(image, cls, scale, region=None):
    pixel_count = image.eq(cls).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=scale,
        maxPixels=1e13
    ).get('DSWE')

    area_km2 = ee.Number(pixel_count).multiply(scale * scale).divide(1e6)
    return pixel_count, area_km2

# DSWE Stats Function
def compute_s2_dswe_stats(image_id, date_str, scale=10, region=None):
    stats = {'date': date_str}
    classes = [0, 1, 2, 3, 4]

    try:
        image = ee.Image(image_id).select('DSWE')

        for cls in classes:
            count, area = calculate_area_from_class(image, cls, scale, region)
            stats[f'count_{cls}'] = count.getInfo() if count else 0
            stats[f'area_km2_{cls}'] = area.getInfo() if area else 0

        water_mask = image.gt(0)
        water_count = water_mask.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=region,
            scale=scale,
            maxPixels=1e13
        ).get('DSWE')

        water_area = ee.Number(water_count).multiply(scale * scale).divide(1e6)
        stats['count_1to4'] = water_count.getInfo() if water_count else 0
        stats['area_km2_1to4'] = water_area.getInfo() if water_area else 0

        return stats

    except Exception as e:
        print(f"⚠️ Error processing {image_id}: {e}")
        return None

# Main Batch Processor
def batch_process_S2_dswe(dswe_folder, start_year, start_month, end_year, end_month, output_csv_path, region=None):
    image_ids = generate_s2_monthly_ids(dswe_folder, start_year, start_month, end_year, end_month)
    results = []

    print(f"🔄 Processing {len(image_ids)} Sentinel-2 DSWE images...")

    for image_id, date_str in tqdm(image_ids):
        stats = compute_s2_dswe_stats(image_id, date_str, scale=10, region=region)
        if stats:
            results.append(stats)

    df = pd.DataFrame(results)
    df.to_csv(output_csv_path, index=False)
    print(f"✅ DSWE summary saved to: {output_csv_path}")

## Execute code for date range

In [2]:
batch_process_LS_dswe_with_qc(
    dswe_folder='projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/DSWE_Products',
    qc_folder='projects/ee-okavango/assets/water_masks/monthly_DSWE_Landsat_30m_v2/QC_Masks',
    start_year=1984,
    start_month=6,
    end_year=2025,
    end_month=7,
    output_csv_path=r'C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\Extents\LS_dswe_monthly_summary_08182025.csv'
)


NameError: name 'batch_process_LS_dswe_with_qc' is not defined

In [10]:
batch_process_S2_dswe(
    dswe_folder='projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products',
    start_year=2017,
    start_month=1,
    end_year=2025,
    end_month=8,
    output_csv_path=r'C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\Extents\S2_dswe_monthly_summary.csv'
)


🔄 Processing 104 Sentinel-2 DSWE images...


  1%|▊                                                                                 | 1/104 [00:00<00:18,  5.67it/s]

⚠️ Error processing projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2017_01: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2017_01' not found (does not exist or caller does not have access).


  2%|█▌                                                                                | 2/104 [00:00<00:16,  6.05it/s]

⚠️ Error processing projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2017_02: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2017_02' not found (does not exist or caller does not have access).


...

⚠️ Error processing projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_09: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_09' not found (does not exist or caller does not have access).
⚠️ Error processing projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_10: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_10' not found (does not exist or caller does not have access).


 22%|█████████████████▉                                                               | 23/104 [00:04<00:15,  5.09it/s]

⚠️ Error processing projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_11: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/monthly_DSWE_Sent2_10m_v2/DSWE_Products/DSWE_2018_11' not found (does not exist or caller does not have access).


100%|██████████████████████████████████████████████████████████████████████████████| 104/104 [1:06:14<00:00, 38.22s/it]

✅ DSWE summary saved to: C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\Extents\S2_dswe_monthly_summary.csv


In [5]:
# Example usage:
folder_prefix = "projects/ee-okavango/assets/water_masks/JASO_DSWE_Landsat_30m_comparison/DSWE_Products_JASO"
df = batch_process_annual_dswe(folder_prefix, 
                               2013, 
                               2029, 
                               r"C:\Users\huckr\Desktop\UCSB\Okavango\Data\Inundation\Extents\dswe_JASO_stats.csv")

📅 Processing 2013...
📅 Processing 2014...
📅 Processing 2015...
📅 Processing 2016...
📅 Processing 2017...
📅 Processing 2018...
📅 Processing 2019...
📅 Processing 2020...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/JASO_DSWE_Landsat_30m_comparison/DSWE_Products_JASO/DSWE_2020_JASO
   Error: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/JASO_DSWE_Landsat_30m_comparison/DSWE_Products_JASO/DSWE_2020_JASO' not found (does not exist or caller does not have access).
📅 Processing 2021...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_masks/JASO_DSWE_Landsat_30m_comparison/DSWE_Products_JASO/DSWE_2021_JASO
   Error: Image.load: Image asset 'projects/ee-okavango/assets/water_masks/JASO_DSWE_Landsat_30m_comparison/DSWE_Products_JASO/DSWE_2021_JASO' not found (does not exist or caller does not have access).
📅 Processing 2022...
⚠️ DSWE image not found or failed to process: projects/ee-okavango/assets/water_